# Explicación Detallada de Pytest para Data Engineers

En este notebook, profundizaremos en los conceptos fundamentales de pytest y cómo aplicarlos específicamente en proyectos de Data Engineering. Exploraremos la sintaxis, características y patrones comunes que hacen de pytest una herramienta tan poderosa para el testing de código relacionado con datos.

## Fundamentos de Pytest

Pytest simplifica enormemente la escritura de tests en Python. A diferencia de frameworks como unittest, que requiere la creación de clases que hereden de `TestCase`, pytest permite escribir tests como simples funciones. Esta simplicidad hace que sea más fácil comenzar a escribir tests y mantenerlos a lo largo del tiempo.

### Instalación de Pytest

Si aún no has instalado pytest, puedes hacerlo fácilmente con pip:

In [ ]:
!pip install pytest pytest-cov

### Estructura básica de un test en Pytest

En pytest, un test es simplemente una función cuyo nombre comienza con `test_`. Dentro de esta función, utilizamos aserciones (`assert`) para verificar que el comportamiento del código es el esperado.

Veamos un ejemplo simple:

In [ ]:
# Ejemplo de una función simple que queremos testear
def calcular_total_con_impuesto(monto, tasa_impuesto=0.16):
    """Calcula el monto total incluyendo impuestos."""
    return monto * (1 + tasa_impuesto)

# Test para la función
def test_calcular_total_con_impuesto():
    # Caso 1: Impuesto por defecto (16%)
    assert calcular_total_con_impuesto(100) == 116.0
    
    # Caso 2: Impuesto personalizado (10%)
    assert calcular_total_con_impuesto(100, 0.10) == 110.0
    
    # Caso 3: Monto cero
    assert calcular_total_con_impuesto(0) == 0.0

### Ejecutando tests

Normalmente, ejecutaríamos los tests desde la línea de comandos con el comando `pytest`. Sin embargo, en un notebook podemos usar el módulo `pytest` directamente:

In [ ]:
# Guardamos la función y el test en un archivo temporal para ejecutarlo
%%file ../tests/temp_test.py
def calcular_total_con_impuesto(monto, tasa_impuesto=0.16):
    """Calcula el monto total incluyendo impuestos."""
    return monto * (1 + tasa_impuesto)

def test_calcular_total_con_impuesto():
    # Caso 1: Impuesto por defecto (16%)
    assert calcular_total_con_impuesto(100) == 116.0
    
    # Caso 2: Impuesto personalizado (10%)
    assert calcular_total_con_impuesto(100, 0.10) == 110.0
    
    # Caso 3: Monto cero
    assert calcular_total_con_impuesto(0) == 0.0

In [ ]:
# Ejecutamos pytest en el archivo temporal
!pytest -xvs temp_test.py

## Características Avanzadas de Pytest

Pytest ofrece muchas características avanzadas que lo hacen especialmente útil para testing en Data Engineering. Veamos algunas de las más importantes:

### 1. Fixtures: Preparación y limpieza del entorno de pruebas

Las fixtures son funciones que pytest ejecuta antes (y opcionalmente después) de los tests para preparar el entorno de pruebas. Son extremadamente útiles en Data Engineering para cargar datasets, configurar conexiones a bases de datos, o preparar cualquier otro recurso necesario para los tests.

Veamos un ejemplo de cómo usar fixtures para cargar nuestro dataset de ventas:

In [ ]:
%%file ../tests/test_with_fixtures.py
import pytest
import pandas as pd

@pytest.fixture
def df_ventas():
    """Fixture que carga el dataset de ventas."""
    return pd.read_csv('../data/ventas_productos.csv')

def calcular_ingresos_por_categoria(df):
    """Calcula los ingresos totales por categoría de producto."""
    return df.groupby('categoria')['total'].sum().to_dict()

def test_calcular_ingresos_por_categoria(df_ventas):
    """Test que verifica el cálculo de ingresos por categoría."""
    # Calculamos los ingresos por categoría
    ingresos = calcular_ingresos_por_categoria(df_ventas)
    
    # Verificamos que todas las categorías esperadas estén presentes
    assert set(ingresos.keys()) == {'Electrónica', 'Accesorios', 'Almacenamiento', 'Componentes', 'Audio', 'Redes', 'Wearables'}
    
    # Verificamos que los ingresos sean números positivos
    for categoria, ingreso in ingresos.items():
        assert ingreso > 0, f"Los ingresos para {categoria} deberían ser positivos"

In [ ]:
# Ejecutamos el test con fixtures
!pytest -xvs test_with_fixtures.py

### 2. Parametrización: Ejecutar el mismo test con diferentes datos

La parametrización es una característica poderosa que permite ejecutar el mismo test con diferentes conjuntos de datos de entrada. Esto es especialmente útil en Data Engineering, donde a menudo necesitamos verificar que nuestras funciones manejen correctamente diferentes tipos de datos o escenarios.

Veamos un ejemplo:

In [ ]:
%%file ../tests/test_parametrized.py
import pytest

def validar_formato_fecha(fecha):
    """Valida que una fecha tenga el formato YYYY-MM-DD."""
    import re
    pattern = r'^\d{4}-\d{2}-\d{2}$'
    return bool(re.match(pattern, fecha))

@pytest.mark.parametrize("fecha,esperado", [
    ("2023-01-05", True),   # Formato correcto
    ("2023-1-5", False),    # Día y mes sin ceros iniciales
    ("01-05-2023", False),  # Formato incorrecto (DD-MM-YYYY)
    ("2023/01/05", False),  # Separadores incorrectos
    ("20230105", False),    # Sin separadores
    ("2023-01-32", True),   # Día inválido pero formato correcto
    ("", False),            # Cadena vacía
])
def test_validar_formato_fecha(fecha, esperado):
    """Test parametrizado para la función de validación de fechas."""
    assert validar_formato_fecha(fecha) == esperado, f"La validación de '{fecha}' debería ser {esperado}"

In [ ]:
# Ejecutamos el test parametrizado
!pytest -xvs test_parametrized.py

### 3. Marcadores: Categorizar y seleccionar tests

Los marcadores permiten categorizar los tests y seleccionar subconjuntos específicos para ejecutar. Esto es útil cuando tenemos diferentes tipos de tests (unitarios, integración, etc.) o cuando algunos tests son lentos o requieren recursos especiales.

Veamos un ejemplo:

In [ ]:
%%file ../tests/test_with_markers.py
import pytest
import pandas as pd
import time

@pytest.fixture
def df_ventas():
    return pd.read_csv('../data/ventas_productos.csv')

@pytest.mark.rapido
def test_columnas_requeridas(df_ventas):
    """Test rápido que verifica que el DataFrame tenga las columnas requeridas."""
    columnas_requeridas = ['id', 'fecha', 'producto', 'categoria', 'precio', 'cantidad', 'descuento', 'total']
    for columna in columnas_requeridas:
        assert columna in df_ventas.columns, f"La columna {columna} debería estar presente"

@pytest.mark.lento
def test_calculo_intensivo(df_ventas):
    """Test lento que simula un cálculo intensivo."""
    # Simulamos un cálculo que toma tiempo
    time.sleep(2)
    
    # Realizamos algún cálculo con el DataFrame
    resultado = df_ventas.groupby(['categoria', 'fecha']).agg({'total': 'sum'}).reset_index()
    assert not resultado.empty, "El resultado no debería estar vacío"

@pytest.mark.validacion
def test_valores_no_negativos(df_ventas):
    """Test que verifica que no haya valores negativos en columnas numéricas."""
    columnas_numericas = ['precio', 'cantidad', 'total']
    for columna in columnas_numericas:
        assert (df_ventas[columna] >= 0).all(), f"No debería haber valores negativos en {columna}"

In [ ]:
# Ejecutamos solo los tests rápidos
!pytest -xvs test_with_markers.py -m rapido

### 4. Fixtures con alcance (scope)

Las fixtures pueden tener diferentes alcances (function, class, module, session) para controlar cuándo se crean y destruyen. Esto es útil para optimizar recursos, especialmente cuando trabajamos con grandes conjuntos de datos o conexiones a bases de datos.

In [ ]:
%%file ../tests/test_fixture_scope.py
import pytest
import pandas as pd
import time

# Esta fixture se ejecutará una vez por sesión de pytest
@pytest.fixture(scope="session")
def df_ventas_session():
    print("\nCargando dataset (scope=session)...")
    time.sleep(1)  # Simulamos una carga lenta
    df = pd.read_csv('../data/ventas_productos.csv')
    print("Dataset cargado (scope=session)")
    return df

# Esta fixture se ejecutará una vez por función de test
@pytest.fixture(scope="function")
def df_ventas_function():
    print("\nCargando dataset (scope=function)...")
    time.sleep(0.5)  # Simulamos una carga lenta
    df = pd.read_csv('../data/ventas_productos.csv')
    print("Dataset cargado (scope=function)")
    return df

def test_1_con_session_fixture(df_ventas_session):
    print("Ejecutando test_1_con_session_fixture")
    assert len(df_ventas_session) > 0

def test_2_con_session_fixture(df_ventas_session):
    print("Ejecutando test_2_con_session_fixture")
    assert 'total' in df_ventas_session.columns

def test_1_con_function_fixture(df_ventas_function):
    print("Ejecutando test_1_con_function_fixture")
    assert len(df_ventas_function) > 0

def test_2_con_function_fixture(df_ventas_function):
    print("Ejecutando test_2_con_function_fixture")
    assert 'total' in df_ventas_function.columns

In [ ]:
# Ejecutamos los tests con fixtures de diferentes alcances
!pytest -xvs test_fixture_scope.py

## Aplicaciones Específicas para Data Engineering

Ahora que hemos visto las características fundamentales de pytest, veamos cómo aplicarlas específicamente en escenarios comunes de Data Engineering.

### 1. Testing de funciones de transformación de datos

En Data Engineering, a menudo escribimos funciones para transformar datos. Pytest nos permite verificar que estas transformaciones produzcan los resultados esperados.

In [ ]:
%%file ../utils/data_processing.py
import pandas as pd
import numpy as np

def calcular_metricas_ventas(df):
    """Calcula métricas de ventas a partir de un DataFrame de ventas.
    
    Args:
        df: DataFrame con columnas 'precio', 'cantidad', 'descuento' y 'total'
        
    Returns:
        dict: Diccionario con métricas calculadas
    """
    metricas = {
        'total_ventas': df['total'].sum(),
        'promedio_venta': df['total'].mean(),
        'total_productos_vendidos': df['cantidad'].sum(),
        'precio_promedio': df['precio'].mean(),
        'descuento_promedio': df['descuento'].mean(),
        'ahorro_total': (df['precio'] * df['cantidad'] * df['descuento']).sum()
    }
    return metricas

def categorizar_productos_por_precio(df):
    """Categoriza productos según su precio.
    
    Args:
        df: DataFrame con columna 'precio'
        
    Returns:
        DataFrame: DataFrame original con columna 'categoria_precio' añadida
    """
    df = df.copy()  # Evitamos modificar el DataFrame original
    
    # Definimos las categorías de precio
    condiciones = [
        (df['precio'] < 50),
        (df['precio'] >= 50) & (df['precio'] < 100),
        (df['precio'] >= 100) & (df['precio'] < 200),
        (df['precio'] >= 200)
    ]
    categorias = ['Económico', 'Estándar', 'Premium', 'Lujo']
    
    # Creamos la nueva columna
    df['categoria_precio'] = np.select(condiciones, categorias, default='Sin categoría')
    
    return df

def calcular_tendencia_ventas(df):
    """Calcula la tendencia de ventas por mes.
    
    Args:
        df: DataFrame con columnas 'fecha' y 'total'
        
    Returns:
        DataFrame: DataFrame con ventas totales por mes y variación porcentual
    """
    # Convertimos la columna fecha a datetime
    df = df.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Extraemos el mes y agrupamos
    df['mes'] = df['fecha'].dt.to_period('M')
    ventas_mensuales = df.groupby('mes')['total'].sum().reset_index()
    ventas_mensuales['mes'] = ventas_mensuales['mes'].astype(str)
    
    # Calculamos la variación porcentual
    ventas_mensuales['variacion_porcentual'] = ventas_mensuales['total'].pct_change() * 100
    
    return ventas_mensuales

In [ ]:
%%file ../tests/test_data_processing.py
import pytest
import pandas as pd
import numpy as np
from utils.data_processing import calcular_metricas_ventas, categorizar_productos_por_precio, calcular_tendencia_ventas

@pytest.fixture
def df_ventas_sample():
    """Fixture que crea un pequeño DataFrame de muestra para testing."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

def test_calcular_metricas_ventas(df_ventas_sample):
    """Test para la función de cálculo de métricas de ventas."""
    metricas = calcular_metricas_ventas(df_ventas_sample)
    
    # Verificamos que todas las métricas esperadas estén presentes
    assert set(metricas.keys()) == {'total_ventas', 'promedio_venta', 'total_productos_vendidos', 
                                    'precio_promedio', 'descuento_promedio', 'ahorro_total'}
    
    # Verificamos algunos cálculos específicos
    assert metricas['total_ventas'] == pytest.approx(1516.94, 0.01)
    assert metricas['total_productos_vendidos'] == 6
    assert metricas['precio_promedio'] == pytest.approx(403.32, 0.01)

def test_categorizar_productos_por_precio(df_ventas_sample):
    """Test para la función de categorización de productos por precio."""
    df_categorizado = categorizar_productos_por_precio(df_ventas_sample)
    
    # Verificamos que se haya añadido la columna de categoría de precio
    assert 'categoria_precio' in df_categorizado.columns
    
    # Verificamos que las categorías sean correctas
    assert df_categorizado.loc[0, 'categoria_precio'] == 'Lujo'  # Laptop HP: 899.99
    assert df_categorizado.loc[1, 'categoria_precio'] == 'Premium'  # Monitor Dell: 249.99
    assert df_categorizado.loc[2, 'categoria_precio'] == 'Estándar'  # Teclado Logitech: 59.99
    
    # Verificamos que el DataFrame original no se haya modificado
    assert 'categoria_precio' not in df_ventas_sample.columns

def test_calcular_tendencia_ventas(df_ventas_sample):
    """Test para la función de cálculo de tendencia de ventas."""
    tendencia = calcular_tendencia_ventas(df_ventas_sample)
    
    # Verificamos que el DataFrame tenga las columnas esperadas
    assert set(tendencia.columns) == {'mes', 'total', 'variacion_porcentual'}
    
    # Verificamos que solo haya un mes (ya que todas las fechas son de enero 2023)
    assert len(tendencia) == 1
    assert tendencia.iloc[0]['mes'] == '2023-01'
    assert tendencia.iloc[0]['total'] == pytest.approx(1516.94, 0.01)
    assert np.isnan(tendencia.iloc[0]['variacion_porcentual'])  # No hay mes anterior para comparar

In [ ]:
# Ejecutamos los tests de procesamiento de datos
!pytest -xvs test_data_processing.py

### 2. Testing de validación de datos

Otra aplicación común en Data Engineering es la validación de datos. Pytest nos permite verificar que nuestras funciones de validación detecten correctamente problemas en los datos.

In [ ]:
%%file ../utils/data_validation.py
import pandas as pd
import numpy as np

def validar_completitud(df, columnas_requeridas=None):
    """Valida que no haya valores nulos en las columnas requeridas.
    
    Args:
        df: DataFrame a validar
        columnas_requeridas: Lista de columnas a verificar. Si es None, se verifican todas.
        
    Returns:
        dict: Diccionario con resultados de la validación
    """
    if columnas_requeridas is None:
        columnas_requeridas = df.columns.tolist()
    
    # Verificamos que todas las columnas requeridas existan
    columnas_faltantes = [col for col in columnas_requeridas if col not in df.columns]
    if columnas_faltantes:
        return {
            'valido': False,
            'error': f"Columnas faltantes: {', '.join(columnas_faltantes)}"
        }
    
    # Verificamos valores nulos
    nulos_por_columna = {col: int(df[col].isnull().sum()) for col in columnas_requeridas}
    tiene_nulos = any(nulos_por_columna.values())
    
    if tiene_nulos:
        columnas_con_nulos = [col for col, nulos in nulos_por_columna.items() if nulos > 0]
        return {
            'valido': False,
            'error': f"Valores nulos encontrados en: {', '.join(columnas_con_nulos)}",
            'detalle': nulos_por_columna
        }
    
    return {
        'valido': True,
        'mensaje': "No se encontraron valores nulos en las columnas requeridas"
    }

def validar_tipos_datos(df, tipos_esperados):
    """Valida que las columnas tengan los tipos de datos esperados.
    
    Args:
        df: DataFrame a validar
        tipos_esperados: Diccionario con columnas y sus tipos esperados
        
    Returns:
        dict: Diccionario con resultados de la validación
    """
    # Verificamos que todas las columnas existan
    columnas_faltantes = [col for col in tipos_esperados.keys() if col not in df.columns]
    if columnas_faltantes:
        return {
            'valido': False,
            'error': f"Columnas faltantes: {', '.join(columnas_faltantes)}"
        }
    
    # Verificamos los tipos de datos
    tipos_incorrectos = {}
    for columna, tipo_esperado in tipos_esperados.items():
        tipo_actual = df[columna].dtype
        if not pd.api.types.is_dtype_equal(tipo_actual, tipo_esperado):
            # Para tipos numéricos, verificamos si podemos convertir sin pérdida de información
            if pd.api.types.is_numeric_dtype(tipo_esperado) and pd.api.types.is_numeric_dtype(tipo_actual):
                continue
            
            # Para fechas, verificamos si podemos convertir
            if tipo_esperado == 'datetime64[ns]' and pd.api.types.is_string_dtype(tipo_actual):
                try:
                    pd.to_datetime(df[columna])
                    continue
                except:
                    pass
            
            tipos_incorrectos[columna] = {'esperado': tipo_esperado, 'actual': str(tipo_actual)}
    
    if tipos_incorrectos:
        return {
            'valido': False,
            'error': f"Tipos de datos incorrectos en {len(tipos_incorrectos)} columnas",
            'detalle': tipos_incorrectos
        }
    
    return {
        'valido': True,
        'mensaje': "Todos los tipos de datos son correctos"
    }

def validar_rango_valores(df, rangos):
    """Valida que los valores estén dentro de los rangos especificados.
    
    Args:
        df: DataFrame a validar
        rangos: Diccionario con columnas y sus rangos (min, max)
        
    Returns:
        dict: Diccionario con resultados de la validación
    """
    # Verificamos que todas las columnas existan
    columnas_faltantes = [col for col in rangos.keys() if col not in df.columns]
    if columnas_faltantes:
        return {
            'valido': False,
            'error': f"Columnas faltantes: {', '.join(columnas_faltantes)}"
        }
    
    # Verificamos los rangos
    fuera_de_rango = {}
    for columna, (min_val, max_val) in rangos.items():
        # Verificamos mínimo
        if min_val is not None:
            valores_bajo_minimo = df[df[columna] < min_val]
            if not valores_bajo_minimo.empty:
                if columna not in fuera_de_rango:
                    fuera_de_rango[columna] = {}
                fuera_de_rango[columna]['bajo_minimo'] = {
                    'cantidad': len(valores_bajo_minimo),
                    'minimo_esperado': min_val,
                    'valor_minimo_encontrado': float(valores_bajo_minimo[columna].min())
                }
        
        # Verificamos máximo
        if max_val is not None:
            valores_sobre_maximo = df[df[columna] > max_val]
            if not valores_sobre_maximo.empty:
                if columna not in fuera_de_rango:
                    fuera_de_rango[columna] = {}
                fuera_de_rango[columna]['sobre_maximo'] = {
                    'cantidad': len(valores_sobre_maximo),
                    'maximo_esperado': max_val,
                    'valor_maximo_encontrado': float(valores_sobre_maximo[columna].max())
                }
    
    if fuera_de_rango:
        return {
            'valido': False,
            'error': f"Valores fuera de rango en {len(fuera_de_rango)} columnas",
            'detalle': fuera_de_rango
        }
    
    return {
        'valido': True,
        'mensaje': "Todos los valores están dentro de los rangos especificados"
    }

In [ ]:
%%file ../tests/test_data_validation.py
import pytest
import pandas as pd
import numpy as np
from utils import validar_completitud, validar_tipos_datos, validar_rango_valores

@pytest.fixture
def df_valido():
    """Fixture que crea un DataFrame válido para testing."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_con_nulos():
    """Fixture que crea un DataFrame con valores nulos."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', None, '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', None],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, None],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_tipos_incorrectos():
    """Fixture que crea un DataFrame con tipos de datos incorrectos."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': ['899.99', '249.99', '59.99'],  # Strings en lugar de float
        'cantidad': [1, 2, 3],
        'descuento': [0.05, 0.00, 0.10],
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

@pytest.fixture
def df_fuera_de_rango():
    """Fixture que crea un DataFrame con valores fuera de rango."""
    data = {
        'id': [1, 2, 3],
        'fecha': ['2023-01-05', '2023-01-10', '2023-01-15'],
        'producto': ['Laptop HP', 'Monitor Dell', 'Teclado Logitech'],
        'categoria': ['Electrónica', 'Electrónica', 'Accesorios'],
        'precio': [899.99, 249.99, 59.99],
        'cantidad': [1, 2, -3],  # Valor negativo
        'descuento': [0.05, 0.00, 1.5],  # Descuento mayor a 1 (100%)
        'total': [854.99, 499.98, 161.97]
    }
    return pd.DataFrame(data)

def test_validar_completitud_sin_nulos(df_valido):
    """Test para validar completitud en un DataFrame sin valores nulos."""
    resultado = validar_completitud(df_valido)
    assert resultado['valido'] is True
    assert 'mensaje' in resultado

def test_validar_completitud_con_nulos(df_con_nulos):
    """Test para validar completitud en un DataFrame con valores nulos."""
    resultado = validar_completitud(df_con_nulos)
    assert resultado['valido'] is False
    assert 'error' in resultado
    assert 'detalle' in resultado
    assert resultado['detalle']['fecha'] == 1
    assert resultado['detalle']['categoria'] == 1
    assert resultado['detalle']['cantidad'] == 1

def test_validar_completitud_columnas_especificas(df_con_nulos):
    """Test para validar completitud solo en columnas específicas."""
    # Validamos solo columnas que no tienen nulos
    resultado = validar_completitud(df_con_nulos, ['id', 'producto', 'precio', 'descuento', 'total'])
    assert resultado['valido'] is True
    
    # Validamos una columna que tiene nulos
    resultado = validar_completitud(df_con_nulos, ['id', 'fecha'])
    assert resultado['valido'] is False
    assert resultado['detalle']['fecha'] == 1

def test_validar_tipos_datos_correctos(df_valido):
    """Test para validar tipos de datos correctos."""
    tipos_esperados = {
        'id': 'int64',
        'precio': 'float64',
        'cantidad': 'int64',
        'descuento': 'float64',
        'total': 'float64'
    }
    resultado = validar_tipos_datos(df_valido, tipos_esperados)
    assert resultado['valido'] is True

def test_validar_tipos_datos_incorrectos(df_tipos_incorrectos):
    """Test para validar tipos de datos incorrectos."""
    tipos_esperados = {
        'id': 'int64',
        'precio': 'float64',  # En df_tipos_incorrectos, precio es string
        'cantidad': 'int64',
        'descuento': 'float64',
        'total': 'float64'
    }
    resultado = validar_tipos_datos(df_tipos_incorrectos, tipos_esperados)
    assert resultado['valido'] is False
    assert 'precio' in resultado['detalle']

def test_validar_rango_valores_dentro_de_rango(df_valido):
    """Test para validar rangos de valores dentro de los límites."""
    rangos = {
        'precio': (0, 1000),
        'cantidad': (0, 10),
        'descuento': (0, 1),
        'total': (0, 1000)
    }
    resultado = validar_rango_valores(df_valido, rangos)
    assert resultado['valido'] is True

def test_validar_rango_valores_fuera_de_rango(df_fuera_de_rango):
    """Test para validar rangos de valores fuera de los límites."""
    rangos = {
        'precio': (0, 1000),
        'cantidad': (0, 10),  # En df_fuera_de_rango, hay un valor negativo
        'descuento': (0, 1),  # En df_fuera_de_rango, hay un descuento > 1
        'total': (0, 1000)
    }
    resultado = validar_rango_valores(df_fuera_de_rango, rangos)
    assert resultado['valido'] is False
    assert 'cantidad' in resultado['detalle']
    assert 'descuento' in resultado['detalle']
    assert 'bajo_minimo' in resultado['detalle']['cantidad']
    assert 'sobre_maximo' in resultado['detalle']['descuento']

In [ ]:
# Ejecutamos los tests de validación de datos
!pytest -xvs test_data_validation.py

## Cobertura de Código

Una métrica importante en testing es la cobertura de código, que mide qué porcentaje de nuestro código está siendo ejecutado por los tests. Pytest, junto con el plugin pytest-cov, nos permite medir y visualizar esta cobertura.

In [ ]:
# Ejecutamos los tests con medición de cobertura
!pytest --cov=utils test_data_processing.py test_data_validation.py

Para obtener un informe más detallado de la cobertura:

In [ ]:
# Generamos un informe detallado de cobertura
!pytest --cov=utils --cov-report=term-missing test_data_processing.py test_data_validation.py

## Integración con Pandas y NumPy

Cuando trabajamos con pandas y NumPy en Data Engineering, hay algunas consideraciones especiales para el testing:

### 1. Comparación de DataFrames

Para comparar DataFrames en tests, podemos usar `pd.testing.assert_frame_equal`:

In [ ]:
%%file ../tests/test_pandas_integration.py
import pytest
import pandas as pd
import numpy as np

def filtrar_por_categoria(df, categoria):
    """Filtra un DataFrame por categoría."""
    return df[df['categoria'] == categoria].reset_index(drop=True)

def test_filtrar_por_categoria():
    # Creamos un DataFrame de prueba
    data = {
        'id': [1, 2, 3, 4],
        'categoria': ['A', 'B', 'A', 'C'],
        'valor': [10, 20, 30, 40]
    }
    df = pd.DataFrame(data)
    
    # Filtramos por categoría 'A'
    resultado = filtrar_por_categoria(df, 'A')
    
    # Creamos el DataFrame esperado
    esperado = pd.DataFrame({
        'id': [1, 3],
        'categoria': ['A', 'A'],
        'valor': [10, 30]
    })
    
    # Comparamos los DataFrames
    pd.testing.assert_frame_equal(resultado, esperado)

def test_filtrar_por_categoria_vacio():
    # Creamos un DataFrame de prueba
    data = {
        'id': [1, 2, 3, 4],
        'categoria': ['A', 'B', 'A', 'C'],
        'valor': [10, 20, 30, 40]
    }
    df = pd.DataFrame(data)
    
    # Filtramos por una categoría que no existe
    resultado = filtrar_por_categoria(df, 'D')
    
    # Verificamos que el resultado sea un DataFrame vacío con las mismas columnas
    assert len(resultado) == 0
    assert list(resultado.columns) == ['id', 'categoria', 'valor']

In [ ]:
# Ejecutamos los tests de integración con pandas
!pytest -xvs test_pandas_integration.py

### 2. Comparación de arrays NumPy

Para arrays de NumPy, podemos usar `np.testing.assert_array_equal` o `np.testing.assert_allclose` para comparaciones con tolerancia:

In [ ]:
%%file ../tests/test_numpy_integration.py
import pytest
import numpy as np

def normalizar_array(arr):
    """Normaliza un array para que sus valores estén entre 0 y 1."""
    min_val = np.min(arr)
    max_val = np.max(arr)
    if min_val == max_val:
        return np.zeros_like(arr)
    return (arr - min_val) / (max_val - min_val)

def test_normalizar_array():
    # Creamos un array de prueba
    arr = np.array([10, 20, 30, 40, 50])
    
    # Normalizamos
    resultado = normalizar_array(arr)
    
    # Creamos el array esperado
    esperado = np.array([0.0, 0.25, 0.5, 0.75, 1.0])
    
    # Comparamos los arrays
    np.testing.assert_allclose(resultado, esperado)

def test_normalizar_array_constante():
    # Creamos un array constante
    arr = np.array([10, 10, 10])
    
    # Normalizamos
    resultado = normalizar_array(arr)
    
    # Creamos el array esperado (todos ceros)
    esperado = np.array([0.0, 0.0, 0.0])
    
    # Comparamos los arrays
    np.testing.assert_array_equal(resultado, esperado)

In [ ]:
# Ejecutamos los tests de integración con numpy
!pytest -xvs test_numpy_integration.py

## Conclusión

En este notebook, hemos explorado en detalle las características y funcionalidades de pytest, con un enfoque específico en su aplicación para proyectos de Data Engineering. Hemos visto:

1. **Fundamentos de pytest**: Sintaxis básica, ejecución de tests y aserciones.
2. **Características avanzadas**: Fixtures, parametrización, marcadores y alcance de fixtures.
3. **Aplicaciones específicas para Data Engineering**: Testing de funciones de transformación y validación de datos.
4. **Cobertura de código**: Medición y visualización de la cobertura de tests.
5. **Integración con pandas y NumPy**: Técnicas específicas para testear código que utiliza estas bibliotecas.

Pytest proporciona un conjunto de herramientas potente y flexible para asegurar la calidad de nuestros proyectos de Data Engineering. Al implementar una estrategia de testing efectiva, podemos tener mayor confianza en la corrección de nuestro código y en la calidad de los datos que procesamos.

En el siguiente notebook, veremos un paso a paso detallado para implementar tests en un proyecto de Data Engineering real, desde la configuración inicial hasta la ejecución y análisis de los resultados.